In [ ]:
import torch
from medcam import medcam
from monai import transforms
import numpy as np
import nibabel as nib
from model import *
from matplotlib import colormaps
from scipy.ndimage import zoom
from util import Util
import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VoxVGG(3).to(device)
# model = VoxResNet().to(device)

# checkpoint_path = r"C:\Users\17993\Downloads\best_epoch17.pth"
# checkpoint = torch.load(checkpoint_path, map_location=torch.device(device))

# # 加载模型权重
# model.load_state_dict(checkpoint['model'])

# # 获取保存的accuracy, loss和epoch
# accuracy = checkpoint['accuracy']
# validation_loss = checkpoint['loss']
# epoch = checkpoint['epoch']
# print(f"Model loaded successfully with accuracy: {100.*accuracy:.2f}%, loss: {validation_loss:.6f}, at epoch: {epoch}")

model = medcam.inject(model, output_dir='attention_maps', backend='gcam', save_maps=False, return_attention=True, layer='conv10')

In [ ]:
model

In [ ]:
transform = transforms.Compose([
    transforms.Resize(spatial_size=[100, 100, 100]),
    transforms.NormalizeIntensity(nonzero=True, channel_wise=True),
])

In [ ]:
img_path = r"C:\Custom\DataSet\ADNI_预处理后\Image\brain_adni_0021_I196077_fsld.nii.gz"
nii_img = nib.load(img_path).get_fdata()
nii_img = nii_img.astype(np.float32)
original_img = nii_img
nii_img = torch.from_numpy(nii_img)
nii_img = nii_img.unsqueeze(0)
nii_img = transform(nii_img)
nii_img = nii_img.unsqueeze(0)
nii_img = nii_img.as_tensor()
print(nii_img.shape)

In [ ]:
nii_img[0][0][40][45][50]

In [ ]:
original_img[40][45][50]

In [ ]:
model.eval()
with torch.no_grad():
    # 进行预测
    outputs, attention_map = model(nii_img)
    # 输出处理，获取预测结果
    _, predicted = torch.max(outputs, 1)
    attention_map = attention_map.detach().cpu().numpy()
attention_map = np.squeeze(attention_map)
print('predicted:', predicted)
print('attention_map shape:', attention_map.shape)

In [ ]:

plt.imsave('./attention_maps/test.jpg', attention_map[40], cmap="jet")

In [ ]:
slice_idx = 50
print(original_img[:,:,slice_idx].shape)
plt.imshow(np.flipud(original_img[:,:,slice_idx].T), cmap='gray')
plt.colorbar()
plt.show()
plt.close()

In [ ]:
cnt = 0
for i in range(110):
    for j in range(110):
        for k in range(110):
            if attention_map[i][j][k] == 0:
                cnt += 1
            else:
                print("test")
print(cnt)

In [ ]:
plt.imshow(np.flipud(attention_map[:,:,45].T), cmap='jet')
plt.colorbar()
plt.show()
plt.close()

In [ ]:
print(attention_map.shape)
print(original_img.shape)

In [ ]:

# 计算新旧尺寸的缩放比例
zoom_factors = np.array((91, 109, 91)) / np.array(attention_map.shape)
# 使用 zoom 函数调整数组大小
attention_map = zoom(attention_map, zoom_factors, order=0)  # order=3 代表三次样条插值
print(attention_map.shape)
print(original_img.shape)

In [ ]:


# 归一化
attention_map = Util.normalize_image_0_to_1(attention_map)
original_img = Util.normalize_image_0_to_1(original_img)
# 获取热力图颜色映射

cmap = colormaps['jet']  # 你可以选择其他颜色映射
attention_map_colored = cmap(attention_map)
plt.imsave('./attention_maps/test.jpg', attention_map[40], cmap="jet")
# 将颜色映射的alpha值设置为热力图的正规化值，以创建透明度效果
# 这允许原始图像通过热力图显示
attention_map_colored[..., -1] = attention_map

In [ ]:
# 创建一个图像来显示结果
fig, ax = plt.subplots()
ax.imshow(original_img[45], vmin=0, vmax=1, cmap="gray")
ax.imshow(attention_map_colored[45], alpha=0.7, vmin=0, vmax=1)  # 调整alpha以改变叠加层的透明度
plt.axis('off')  # 关闭坐标轴

In [ ]:
def overlap(image:np.ndarray, attention_map:np.ndarray, alpha=0.7):
    # 缩放热力图适应原图
    zoom_factors = np.array((image.shape[0], image.shape[1], image.shape[2])) / np.array(attention_map.shape)
    attention_map = zoom(attention_map, zoom_factors, order=3, mode="reflect")  # order=3 代表三次样条插值
    
    # 归一化
    image = Util.normalize_image_0_to_1(image)
    attention_map = Util.normalize_image_0_to_1(attention_map)
    
    # 处理热力图
    cmap = colormaps['jet']  # 你可以选择其他颜色映射
    attention_map_colored = cmap(attention_map)
    attention_map_colored[..., -1] = attention_map
    attention_map = attention_map_colored
    
    # 将灰度图转换为 RGB
    image_rgb = np.stack([image] * 3, axis=-1)
    # 提取 RGBA 热力图的颜色和 alpha 通道
    color_layer = attention_map[..., :3]
    alpha_layer = attention_map[..., 3:]
    # 使用热力图的 alpha 通道来混合原图和热力图的颜色
    overlap_image = (alpha * alpha_layer) * color_layer + (1 - (alpha_layer * alpha)) * image_rgb
    return overlap_image

In [ ]:
overlap_image = overlap(original_img, attention_map, alpha=0.7)
print(overlap_image.shape)
plt.imshow(overlap_image[45], vmin=0, vmax=1)
plt.show()

In [ ]:
plt.imshow(original_img[45], alpha=1, vmin=0, vmax=1, cmap="gray")

In [ ]:
import numpy as np

# 假设 original_img 和 attention_map_colored 已经定义

# 将灰度图转换为 RGB
original_img_rgb = np.stack([original_img]*3, axis=-1)

# 计算叠加图像
# 提取 RGBA 热力图的颜色和 alpha 通道
color_layer = attention_map_colored[..., :3]
alpha_layer = attention_map_colored[..., 3:]

# 使用热力图的 alpha 通道来混合原图和热力图的颜色
combined_img = 0 * alpha_layer * color_layer + (1 - alpha_layer * 0) * original_img_rgb

# combined_img 现在是叠加了热力图的 RGB 图像
plt.imshow(combined_img[45], vmin=0, vmax=1)

In [1]:
import os
import shutil

# 指定要清空的文件夹路径
folder_path = r'C:\Users\17993\Desktop\test\vector'

# 遍历文件夹中的所有文件和子文件夹
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        # 如果是文件，则删除
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.remove(file_path)
        # 如果是目录，则删除目录及其所有内容（可选）
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print(f'Failed to delete {file_path}. Reason: {e}')